In [13]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from xgboost import XGBRegressor

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
train = pd.read_csv('data/train.csv').drop('Id', axis=1)
test = pd.read_csv('data/test.csv').drop('Id', axis=1)

In [ ]:
for i in train.select_dtypes(include=['number']).columns:
    train[i] = train[i].fillna(-1)
    try: # because of SalePrice
        test[i] = test[i].fillna(-1)
    except:
        pass

for i in train.select_dtypes(include=['object']).columns:
    train[i] = train[i].fillna('')
    test[i] = test[i].fillna('')

In [ ]:
(train.Exterior1st == train.Exterior2nd).sum()
train['isExteriorSame'] = (train.Exterior1st == train.Exterior2nd).astype(int)
test['isExteriorSame'] = (test.Exterior1st == test.Exterior2nd).astype(int)
train.Utilities.value_counts()

train = train.drop('Utilities', axis=1)
test = test.drop('Utilities', axis=1)


In [ ]:
le_columns = ['Alley', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition', 'Fence', 'MiscFeature', 'PoolQC', 'KitchenQual']

In [ ]:
label_encoder = LabelEncoder()
for i in le_columns:
    train[i] = label_encoder.fit_transform(train[i])
    test[i] = label_encoder.fit_transform(test[i])
train.head()

In [ ]:
train.dtypes.value_counts() # all is ok

In [ ]:
X = train.drop('SalePrice', axis=1)
y = train.SalePrice

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
model = RandomForestRegressor(100)
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test) # not bad score

In [ ]:
mse(y_test, model.predict(X_test)) ** 0.5

In [ ]:
cb_model = CatBoostRegressor(5000)
cb_model.fit(X_train, y_train, verbose=False)

In [ ]:
mse(y_test, cb_model.predict(X_test)) ** 0.5

In [ ]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train, verbose=False)


In [ ]:
xgb_model.score(X_test, y_test)

In [ ]:
mse(y_test, xgb_model.predict(X_test)) ** 0.5

In [ ]:
rf_preds = model.predict(test)
cb_preds = cb_model.predict(test)
xgb_preds = xgb_model.predict(test)
preds = xgb_preds * 0.1 + cb_preds * 0.8 + rf_preds * 0.1

In [ ]:
pd.DataFrame({
    'Id': [i for i in range(1461, 2920)],
    'SalePrice': preds
}).to_csv('submit-rf.csv', index=False)